In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

import zipfile

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
with zipfile.ZipFile('/kaggle/input/dogs-vs-cats/train.zip',mode='r') as z:
    z.extractall(".")
with zipfile.ZipFile('/kaggle/input/dogs-vs-cats/test1.zip',mode='r') as z:
    z.extractall(".")

In [ ]:
import cv2
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.models import Sequential 
from tensorflow.keras.layers import Dense,Conv2D,MaxPooling2D,Activation,Flatten,Dropout

In [ ]:
file_store='/kaggle/working/train'

In [ ]:
from matplotlib.image import imread
for i in range(9):
    plt.subplot(330+1+i)
    filename=file_store+'/dog.'+str(i)+'.jpg'
    image=imread(filename)
    plt.imshow(image)
plt.show()    

In [ ]:
from matplotlib.image import imread
for i in range(9):
    plt.subplot(330+1+i)
    filename=file_store+'/cat.'+str(i)+'.jpg'
    image=imread(filename)
    plt.imshow(image)
plt.show()   

In [ ]:
for p in os.listdir(file_store):
    category=p.split('.')[0]
    img_array=cv2.imread(os.path.join(file_store,p),cv2.IMREAD_GRAYSCALE)
    new_img_array=cv2.resize(img_array,dsize=(120,120))
    plt.imshow(new_img_array,cmap='gray')
    break

In [ ]:
X=[]
Y=[]
convert=lambda category : int(category=='dog')
def create_test_data(file_store):
    for p in os.listdir(file_store):
        category=p.split('.')[0]
        category=convert(category)
        img_array=cv2.imread(os.path.join(file_store,p),cv2.IMREAD_GRAYSCALE)
        new_img_array=cv2.resize(img_array,dsize=(120,120))
        X.append(new_img_array)
        Y.append(category)

In [ ]:
file_store_test='/kaggle/working/test1/'
X_test=[]
id_line=[]
def create_test1_data(file_store_test):
    for p in os.listdir(file_store_test):
        id_line.append(p.split('.')[0])
        img_array=cv2.imread(os.path.join(file_store_test,p),cv2.IMREAD_GRAYSCALE)
        new_img_array=cv2.resize(img_array,dsize=(120,120))
        X_test.append(new_img_array)


Training Data

In [ ]:
create_test_data(file_store)
X=np.array(X).reshape(-1,120,120,1)
Y=np.array(Y)
X=X/255.0
X.shape[1:]

Testing Data

In [ ]:
create_test1_data(file_store_test)
X_test = np.array(X_test).reshape(-1,120,120,1)
X_test = X_test/255

In [ ]:
model=Sequential()
model.add(Conv2D(64,(3,3),activation='relu',input_shape=X.shape[1:]))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(64,(3,3),activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(64,(3,3),activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Flatten())

model.add(Dense(64,activation='relu'))
model.add(Dense(1,activation='sigmoid'))

model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

In [ ]:
model.summary()

In [ ]:
model.fit(X,Y,batch_size=32,epochs=10,validation_split=0.2)

In [ ]:
predictinos=model.predict(X_test)

In [ ]:
predicted_val = [int(round(p[0])) for p in predictinos]

In [ ]:
submission_df = pd.DataFrame({'id':id_line, 'label':predicted_val})

In [ ]:
submission_df.to_csv("submission.csv", index=False)